<a href="https://colab.research.google.com/github/heriyang9000/test2/blob/main/test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import getpass
import pandas as pd
import ipywidgets as widgets

In [251]:
import panel as pn
pn.extension()

password_input = pn.widgets.PasswordInput(name='Password Input', placeholder='Enter a string here...')
password_input

widgets.Password(
    value='',
    placeholder='Enter password',
    description='Password:',
    disabled=False
)

Password(description='Password:', placeholder='Enter password')

In [253]:
print(widgets.Password.value)

In [238]:
multi_select = pn.widgets.MultiSelect(name='MultiSelect', value=['Apple', 'Pear'],
    options=['Apple', 'Banana', 'Pear', 'Strawberry'], size=8)
multi_select

MultiSelect(options=['Apple', 'Banana', ...], size=8, value=['Apple', 'Pear'])

In [142]:
initial_equity=input()
bidaskspread=input()
commision=input()
interest=input()

100
10
10
10


In [254]:
exchange = 'NASDAQ'
drop_down = widgets.Dropdown(options=['NASDAQ','NYSE','LSE','EURONEXT'],
                                description='Choose',
                                disabled=False)

def dropdown_handler(change):
    global exchange
    # print(change.new)
    exchange = change.new
drop_down.observe(dropdown_handler, names='value')
display(drop_down)

Dropdown(description='Choose', options=('NASDAQ', 'NYSE', 'LSE', 'EURONEXT'), value='NASDAQ')

In [255]:
# print(exchange)
if exchange == 'LSE':
    ticker = 'a'
    drop_down = widgets.SelectMultiple(
    options=['a','b','c','d','e','f','g','h','i','j','k','l'],
    value=['a'],
    rows=10,
    description='Fruits',
    disabled=False
) 

    def dropdown_handler(change):
        global ticker
        # print(change.new)
        ticker = change.new
    drop_down.observe(dropdown_handler, names='value')
    display(drop_down)
else:
  print("no exchange select")

SelectMultiple(description='Fruits', index=(0,), options=('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k…

In [145]:
widgets.BoundedFloatText(
    value=7.5,
    min=0,
    max=10.0,
    step=0.1,
    description='Text:',
    disabled=False
)

BoundedFloatText(value=7.5, description='Text:', max=10.0, step=0.1)

In [185]:
widgets.Checkbox(
    value=False,
    description='Check me',
    disabled=False,
    indent=False
)

Checkbox(value=False, description='Check me', indent=False)

In [ ]:
df01 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df02 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df03 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df04 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df05 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df06 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df07 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df08 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df09 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df10 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df11 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df12 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df13 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df14 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df15 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df16 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df17 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df18 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df19 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df20 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df21 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df22 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df23 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df24 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df25 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df26 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df27 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df28 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df29 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df30 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')

print(df01.to_string()) 